In [1]:
# Generic imports
import pandas as pd
import plotly
import time
import logging

# 5G visualization logic
import trace_visualizer
import trace_plotting

In [2]:
# Takes as input a N3 and N6 capture with UDP pseudo-random paylods (e.g. as used by Spirent) and plots one-way delays based on the the results of each packet
n3_capture = 'doc/n3_example.pcap'
n6_capture = 'doc/n6_example.pcap' # I don't have an example but you should put here a capture from the N6 interface
output_file = 'doc/UP_example_analysis'

# Needed to separate UE and DN flows
application_ip = '172.16.11.2'

In [6]:
http2_ports = "32445,5002,5000,32665,80,32077,5006,8080,3000,8081"
wireshark_version = '4.2.3'

trace_visualizer.application_logger.setLevel(logging.INFO) # Change this if you want more info
n3_pdml_file = trace_visualizer.call_wireshark(wireshark_version,"OSX", n3_capture, http2_ports, mode='GTP', check_if_exists=True)[0]
n6_pdml_file = trace_visualizer.call_wireshark(wireshark_version, "OSX",n6_capture, http2_ports, mode='UDP', check_if_exists=True)[0]

['4.2.3']
['doc/n3_example.pcap']
asdkjhasdkj checkin /Applications/Wireshark.app/Contents/MacOS/tshark
['4.2.3']
['doc/n6_example.pcap']
asdkjhasdkj checkin /Applications/Wireshark.app/Contents/MacOS/tshark


In [7]:
start_time = time.perf_counter()
n3_packets = trace_plotting.read_xml_file_line_basis(n3_pdml_file)
end_time = time.perf_counter()
total_time = round(end_time - start_time, 2)
print(f'N3: {len(n3_packets)} packets parsed. Total time taken: {total_time}s')

start_time = time.perf_counter()
n6_packets = trace_plotting.read_xml_file_line_basis(n6_pdml_file)
end_time = time.perf_counter()
total_time = round(end_time - start_time, 2)
print(f'N6: {len(n6_packets)} parsed. Total time taken: {total_time}')

N3: 92 packets parsed. Total time taken: 0.05s
N6: 91 parsed. Total time taken: 0.02


In [8]:
# Classification of packets and join of the data from each interface
packets = n3_packets.set_index('udp.payload').join(n6_packets.set_index('udp.payload'), how='inner', lsuffix='.n3', rsuffix='.n6')
packets['flow'] = ''

packets_from_application = packets['ip.src.n6']==application_ip
packets_to_application = packets['ip.dst.n6']==application_ip

packets.loc[packets_from_application,'flow'] = 'DN->UE'
packets.loc[packets_to_application,'flow'] = 'UE->DN'
packets.reset_index(inplace=True)

# We will have positive and negative values because we have packets in both directions
packets['delay_us'] = ((packets['timestamp.n6'] - packets['timestamp.n3'])*1000000).abs() # Timestmp is in seconds

display(packets)

,udp.payload,timestamp.n3,ip.src.n3,ip.dst.n3,udp.srcport.n3,udp.dstport.n3,protocol_count.n3,frame_nr.n3,timestamp.n6,ip.src.n6,ip.dst.n6,udp.srcport.n6,udp.dstport.n6,protocol_count.n6,frame_nr.n6,flow,delay_us
0,000000016050c3da00092fcb0000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,1,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,1,UE->DN,3937.005997
1,000000026050c3da0009338a0000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,2,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,3,UE->DN,3781.080246
2,000000036050c3da0009376c0000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,3,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,5,UE->DN,3782.033920
3,000000046050c3da00093b520000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,4,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,7,UE->DN,3761.053085
4,000000056050c3da00093f350000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,5,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,9,UE->DN,3787.994385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0000002a6050c3da0009cfd300000ea000000000000000...,1.615906e+09,172.16.11.2,10.0.255.2,2003,2002,5,83,1.615906e+09,172.16.11.2,10.200.9.70,2003,1029,2,76,DN->UE,3770.112991
75,000000326050c3da0009eefc0000000000000000000000...,1.615906e+09,10.0.255.2,172.16.11.2,2002,2003,5,84,1.615906e+09,10.200.9.70,172.16.11.2,1029,2003,2,90,UE->DN,3767.013550
76,0000002b6050c3da0009d3ab00000e8800000000000000...,1.615906e+09,172.16.11.2,10.0.255.2,2003,2002,5,85,1.615906e+09,172.16.11.2,10.200.9.70,2003,1029,2,78,DN->UE,3737.211227
77,0000002c6050c3da0009d79d00000ea500000000000000...,1.615906e+09,172.16.11.2,10.0.255.2,2003,2002,5,87,1.615906e+09,172.16.11.2,10.200.9.70,2003,1029,2,80,DN->UE,3710.985184


In [9]:
print(f'N3 packets: {len(n3_packets):,} packets')
print(f'N6 packets: {len(n6_packets):,} packets')
print(f'Joined list: {len(packets):,} packets') 

N3 packets: 92 packets
N6 packets: 91 packets
Joined list: 79 packets


In [10]:
import plotly.graph_objects as go

bin_size = 20
packets_from_application = packets['flow']=='DN->UE'
packets_to_application = packets['flow']=='UE->DN'

# It doesn't make much sense to plot with much higher accuracy than 50us
hist_array, hist_bins = trace_plotting.get_histogram_data(packets.loc[:,'delay_us'], bin_size)
histogram_line = go.Scatter(
        x=hist_bins,
        y=hist_array,
        mode='lines',
        name='',
        line_shape='spline',
        opacity=0.75,
        showlegend=False)

fig = go.Figure(data=[histogram_line])
fig.update_xaxes(title_text='Delay (μs)')
fig.show()

In [11]:
# Save data
data_to_pickle = {}
data_to_pickle['packets'] = packets
data_to_pickle['packets_n3'] = n3_packets
data_to_pickle['packets_n6'] = n6_packets
data_to_pickle['histogram'] = (hist_array, hist_bins)
data_to_pickle['histogram_bin_size'] = bin_size
trace_visualizer.application_logger.setLevel(logging.DEBUG)
trace_plotting.compressed_pickle(output_file, data_to_pickle)

DEBUG:root:Saving data to doc/UP_example_analysis.pbz2
